NOT using any Yahoo! data so as to not overfit (i.e. cheat) for the final evaluation.

In [1]:
from __future__ import print_function
import numpy as np

In [2]:
np.random.seed(0xF00BA2)

In [3]:
# Where we had our Scala program dump all the training data.
# (back to python, to main, to src, to root)

csv_file_name = "../../../../data/all-candidates-05-21-18-44.csv"

In [4]:
# Load our data loading and scaling utilities.
%run '../data_util.py'

In [5]:
FEATURE_COUNT = 24

X_raw, y_raw = load_training_data(csv_file_name, FEATURE_COUNT)

Import was successful.
Feature shape: (83098, 24)
Label shape: (83098,)


In [6]:
print("Raw feature max vals: {0}".format(np.max(X_raw, axis=0)))
print("Raw feature min vals: {0}".format(np.min(X_raw, axis=0)))
print("Raw feature mean vals: {0}".format(np.mean(X_raw, axis=0)))
print("Raw feature stds: {0}".format(np.std(X_raw, axis=0)))

print()
print("Unwanted value checks:")
print("NaNs: {0}".format(np.sum(np.isnan(X_raw), axis=0)))
print("Infinities: {0}".format(np.sum(np.isinf(X_raw), axis=0)))

Raw feature max vals: [  3.39000000e+08   1.00000000e+02   2.53333333e+01   2.53333333e+01
   1.00000000e+04   3.66000000e+02   3.10000000e+01   1.00000000e+00
   2.50000000e+01   1.00000000e+00   1.00000000e+00   1.00000000e+00
   1.00000000e+00   1.00000000e+00   5.00000000e-01   5.00000000e-01
   5.00000000e-01   1.70000000e+01   1.70000000e+01   6.42652725e+01
   2.50000000e+01   7.90000000e+01   1.00000000e+00   1.00000000e+00]
Raw feature min vals: [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.]
Raw feature mean vals: [  1.81623980e+07   9.41403042e+01   4.04306277e+00   3.94101871e+00
   1.01245267e+01   1.37549821e+02   6.60692931e+00   1.55968254e-01
   2.29000023e+01   1.86788303e-01   2.26335283e-01   5.49596488e-01
   5.96867384e-01   5.74718401e-01   2.34007299e-01   2.63707986e-01
   2.44189753e-01   1.96623179e+00   2.68218218e+00   1.56912862e+01
   5.18239594e+00   1.69698067e+01   9.92764596e-03   4.07253277e-02]
Raw 

In [7]:
import sklearn
X_res, y_res, _, _, _ = rescale(X_raw, y_raw)
X, y = sklearn.utils.shuffle(X_res, y_res)

[ 0.51073793 -0.19056737  0.2679928  ..., -0.53767211 -0.63895775
 -0.68314472]
[ 3.18396709  1.61860785  2.82324368 ...,  0.90721663  0.77648483
  0.80031476]


In [8]:
pos_count = np.sum(y == 1)
neg_count = np.sum(y == 0)
print("We have {0} positive labels.".format(pos_count))
print("We have {0} negative labels.".format(neg_count))

We have 622 positive labels.
We have 82476 negative labels.


## Train/validation split with slicing wizardry

In [10]:
# Use a much smaller ratio of negative to positive samples in the validation
# set, for more accurate validation results.
VALIDATION_NEG_TO_POS_RATIO = 1

# The number of data points in the validation dataset, as a function of the
# number of positive labels. Necessary because of the large imbalance between
# our positive and negative training examples.
VALIDATION_PERCENT_OF_POSITIVE_LABELS = 0.25

pos_count_valid = int(pos_count * VALIDATION_PERCENT_OF_POSITIVE_LABELS)
neg_count_valid = pos_count_valid * VALIDATION_NEG_TO_POS_RATIO

# Indexes of positive rows to use for validation.
# This witchcraft isolates the indexes of the first 'pos_count_valid' rows
# with positive labels in the training data.
y_pos_ind = (y == 1)
y_pos_counts = np.cumsum(y_pos_ind)
y_pos_lim = np.where(y_pos_counts == (pos_count_valid + 1))[0][0]
y_pos_ind[y_pos_lim:] = False

# This bit does the same but for the first 'neg_count_valid' rows with negative
# labels.
y_neg_ind = (y == 0)
y_neg_counts = np.cumsum(y_neg_ind)
y_neg_lim = np.where(y_neg_counts == (neg_count_valid + 1))[0][0]
y_neg_ind[y_neg_lim:] = False

# Make sure that there's no overlap, which would signify that we messed something
# up with the slicing/indexing.
assert np.sum(y_pos_ind & y_neg_ind) == 0

X_valid = X[y_pos_ind | y_neg_ind]
y_valid = y[y_pos_ind | y_neg_ind]

X_train = X[~(y_pos_ind | y_neg_ind)]
y_train = y[~(y_pos_ind | y_neg_ind)]

# # Just some manual extra checks.
# # TODO(andrei): Label better or remove.
print(np.sum(y_pos_ind))
print(np.sum(y_neg_ind))
print("Training shape:")
print(X_train.shape)
print(y_train.shape)
print("Validation shape + sanity check:")
print(X_valid.shape)
print(y_valid.shape, np.sum(y_valid == 1), np.sum(y_valid == 0))
print(pos_count_valid, neg_count_valid, pos_count_valid + neg_count_valid)

155
155
Training shape:
(82788, 24)
(82788,)
Validation shape + sanity check:
(310, 24)
(310,) 155 155
155 155 310


### Paranoid version

In [12]:
pos_ids = np.where(y == 1)[0]
neg_ids = np.where(y == 0)[0]
print("Pos/neg shapes (all data):")
print(pos_ids.shape)
print(neg_ids.shape)

# Partition the data in the old-fashioned way
pos_count_train = pos_count - pos_count_valid
neg_count_train = neg_count - neg_count_valid
pos_idx = 0
neg_idx = 0
features= X.shape[1]
x_pos_valid = np.zeros((pos_count_valid, features))
x_neg_valid = np.zeros((neg_count_valid, features))
x_pos_train = np.zeros((pos_count_train, features))
x_neg_train = np.zeros((neg_count_train, features))

y_pos_valid = np.zeros(pos_count_valid)
y_neg_valid = np.zeros(neg_count_valid)
y_pos_train = np.zeros(pos_count_train)
y_neg_train = np.zeros(neg_count_train)
for idx in range(y.shape[0]):
    if y[idx] == 1:
        # Positive label
        if pos_idx < pos_count_valid:
            # Still adding to validation dataset
            x_pos_valid[pos_idx] = X[idx]
            y_pos_valid[pos_idx] = y[idx]
            pos_idx += 1
        else:
            # Done with validation. Now adding to training dataset.
            x_pos_train[pos_idx - pos_count_valid] = X[idx]
            y_pos_train[pos_idx - pos_count_valid] = y[idx]
            pos_idx += 1
    elif y[idx] == 0:
        # Negative label
        if neg_idx < neg_count_valid:
            x_neg_valid[neg_idx] = X[idx]
            y_neg_valid[neg_idx] = y[idx]
            neg_idx += 1
        else:
            x_neg_train[neg_idx - neg_count_valid] = X[idx]
            y_neg_train[neg_idx - neg_count_valid] = y[idx]
            neg_idx += 1
    else:
        raise ValueError("Invalid y value: {}".format(y[idx]))

print("Positive: train/valid")
print(pos_count_train, pos_count_valid)
print("Negative: train/valid")
print(neg_count_train, neg_count_valid)
print(x_neg_valid.shape)
print(x_neg_valid[-1, :])
print(x_pos_valid.shape)
print(x_pos_valid[-1, :])

# Uncomment these to actually use the results of this cell later on!
# X_valid = np.append(x_neg_valid, x_pos_valid, axis=0)
# X_train = np.append(x_neg_train, x_pos_train, axis=0)
# y_valid = np.append(y_neg_valid, y_pos_valid, axis=0)
# y_train = np.append(y_neg_train, y_pos_train, axis=0)
# X_valid, y_valid = sklearn.utils.shuffle(X_valid, y_valid)
# X_train, y_train = sklearn.utils.shuffle(X_train, y_train)

Pos/neg shapes (all data):
(622,)
(82476,)
Positive: train/valid
467 155
Negative: train/valid
82321 155
(155, 24)
[ 0.17658184  0.27414953 -1.00096898 -0.92371079 -0.03186035  0.38600132
 -1.1710565  -0.52652172  0.46199597 -0.12856004 -0.24175296  0.46905942
  0.36299422  0.41860687  0.0742085  -0.0631984   0.02733468 -0.51809587
 -0.33275335 -1.36190716 -1.16110862 -1.39174411 -0.11323911  3.12682712]
(155, 24)
[-0.06647803  0.27414953  3.40845051  3.35929641 -0.03186035  0.32762722
  0.76740132  0.74474069 -0.63117935 -0.50888716  0.10439152 -0.06307559
  0.84405396  0.79168007 -0.98710201  0.32099611 -0.88558805  1.98418232
  4.05724722 -0.55683927  2.95108517 -0.57784553  0.18019221  0.67657722]


## Logistic Regression

In [24]:
from operator import itemgetter

def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("[{0}] Mean validation score: {1:.3f} (std: {2:.3f}): {3}".format(
              i,
              score.mean_validation_score,
              np.std(score.cv_validation_scores),
              score.parameters))

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *


lr_clf = LogisticRegression(penalty='l2', C=0.1, class_weight='balanced')
lr_clf.fit(X_train, y_train)

lr_pred = lr_clf.predict(X_valid)
print("Confusion matrix:\n", confusion_matrix(y_valid, lr_pred))
print("F1:       ", f1_score(y_valid, lr_pred))
print("Precision:", precision_score(y_valid, lr_pred))
print("Recall:   ", recall_score(y_valid, lr_pred))

Confusion matrix:
 [[143  12]
 [ 12 143]]
F1:        0.922580645161
Precision: 0.922580645161
Recall:    0.922580645161


In [26]:
from sklearn.grid_search import GridSearchCV
from sklearn.externals.joblib import delayed

# Simple cross-validation doesn't work well out of the box, as our data is very imbalanced.
# score_on_validation = lambda est, xx_ignore, yy_ignore: est.score(X_valid, y_valid)
def score_on_validation(est, xx_ignore, yy_ignore):
    return est.score(X_valid, y_valid)

In [35]:
pg_lr = {
    'C': [0.0001, 0.0005, 0.001, 0.005, 0.1, 0.5, 1, 5],
    'penalty': ['l1', 'l2'],
}

# The CV isn't really necessary, nor does it make sense, but there's no real way to avoid it.
gs_lr = GridSearchCV(lr_clf, pg_lr, cv=2, scoring=score_on_validation, n_jobs=-1)

In [36]:
gs_lr.fit(X_train, y_train)

GridSearchCV(cv=2, error_score='raise',
       estimator=LogisticRegression(C=0.1, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [0.0001, 0.0005, 0.001, 0.005, 0.1, 0.5, 1, 5]},
       pre_dispatch='2*n_jobs', refit=True,
       scoring=<function score_on_validation at 0x1101a1f28>, verbose=0)

In [37]:
report(gs_lr.grid_scores_, n_top=100)

[0] Mean validation score: 0.932 (std: 0.006): {'penalty': 'l2', 'C': 0.0005}
[1] Mean validation score: 0.931 (std: 0.005): {'penalty': 'l2', 'C': 0.001}
[2] Mean validation score: 0.927 (std: 0.005): {'penalty': 'l1', 'C': 0.005}
[3] Mean validation score: 0.926 (std: 0.006): {'penalty': 'l2', 'C': 0.005}
[4] Mean validation score: 0.921 (std: 0.008): {'penalty': 'l1', 'C': 0.5}
[5] Mean validation score: 0.921 (std: 0.008): {'penalty': 'l2', 'C': 0.5}
[6] Mean validation score: 0.921 (std: 0.008): {'penalty': 'l1', 'C': 1}
[7] Mean validation score: 0.919 (std: 0.010): {'penalty': 'l2', 'C': 1}
[8] Mean validation score: 0.919 (std: 0.010): {'penalty': 'l1', 'C': 5}
[9] Mean validation score: 0.919 (std: 0.010): {'penalty': 'l2', 'C': 5}
[10] Mean validation score: 0.919 (std: 0.006): {'penalty': 'l1', 'C': 0.1}
[11] Mean validation score: 0.919 (std: 0.006): {'penalty': 'l2', 'C': 0.1}
[12] Mean validation score: 0.915 (std: 0.002): {'penalty': 'l2', 'C': 0.0001}
[13] Mean validati

## Linear SVC

In [38]:
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC, SVR, LinearSVC, LinearSVR

# The "class_weight='balanced'" param tells the SVM to give adaptive weights
# to the labels from each class, in order to account for imbalanced data.

vanilla_clf = SGDClassifier(class_weight='balanced', n_iter=5)
print("Will score and then fit the following linear classifier on the training data.")
print(vanilla_clf)
print("Shape of X_train: {}".format(X_train.shape))
print("Shape of X_valid: {}".format(X_valid.shape))
scores = cross_val_score(vanilla_clf, X_train, y_train, cv=10)

print("Naive CV scores (misleading due to data imbalance):")
print(scores)
print(np.mean(scores))

vanilla_clf.fit(X_train, y_train)

print("On validation data")
# vanilla_val_y = vanilla_clf.predict(X_valid)
y_valid_pos_mask = (y_valid == 1)
# print(np.sum(y_valid_pos_mask))
X_valid_pos = X_valid[y_valid_pos_mask]
print(X_valid_pos.shape)

# We are only validating positive labels.
vanilla_y_pos_predict = vanilla_clf.predict(X_valid_pos)
vanilla_tp = np.sum(vanilla_y_pos_predict == 1)
print("True positives")
print(vanilla_tp)
print("False negatives")
print(np.sum(vanilla_y_pos_predict == 0))

vanilla_pos_acc = vanilla_tp / vanilla_y_pos_predict.shape[0]
print("Accuracy on positive data: {0:6.4f}".format(vanilla_pos_acc))
    
print("Confusion matrix as an extra sanity check:")
vclf_pred = vanilla_clf.predict(X_valid)
print(confusion_matrix(y_valid, vclf_pred))
print("Classifier (!) F1-score:")
print(f1_score(y_valid, vclf_pred))
print("Precision:", precision_score(y_valid, vclf_pred))
print("Recall:", recall_score(y_valid, vclf_pred))

Will score and then fit the following linear classifier on the training data.
SGDClassifier(alpha=0.0001, average=False, class_weight='balanced',
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)
Shape of X_train: (82788, 24)
Shape of X_valid: (310, 24)
Naive CV scores (misleading due to data imbalance):
[ 0.94408213  0.9583283   0.92269598  0.92559488  0.92317913  0.90940935
  0.86520111  0.92196183  0.89997584  0.87798985]
0.914841838935
On validation data
(155, 24)
True positives
144
False negatives
11
Accuracy on positive data: 0.9290
Confusion matrix as an extra sanity check:
[[136  19]
 [ 11 144]]
Classifier (!) F1-score:
0.905660377358
Precision: 0.883435582822
Recall: 0.929032258065


Sample for train-A:
```
[ 0.59212345  0.69622294  0.93390143  0.69415016  0.73952096  0.71020502
  0.73324119  0.67396313  0.69746544  0.78225806]
0.725305177786
```

In [39]:
lin_clf = SGDClassifier(class_weight='balanced', alpha=0.001, loss='hinge')
# lin_clf = LinearSVC() -> validation accuracy 0.5 !!
lin_clf.fit(X_train, y_train)

SGDClassifier(alpha=0.001, average=False, class_weight='balanced',
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [40]:
score = lin_clf.score(X_valid, y_valid)
y_valid_predicted = lin_clf.predict(X_valid)

print("Validation data shape: {0}".format(X_valid.shape))
print("Validation data neg:pos ratio: {0}".format(VALIDATION_NEG_TO_POS_RATIO))
print("Validation accuracy: {0}".format(score))

print(confusion_matrix(y_valid, y_valid_predicted))

Validation data shape: (310, 24)
Validation data neg:pos ratio: 1
Validation accuracy: 0.9096774193548387
[[134  21]
 [  7 148]]


Sample from train-AB with fewer features (20 only) and old WAT style, salsa-auth:
```
Validation data shape: (266, 20)
Validation data neg:pos ratio: 1
Validation accuracy: 0.8421052631578947
[[115  18]
 [ 24 109]]
 ```

Sample from train-A only:

```
Validation data shape: (136, 10)
Validation data neg:pos ratio: 1
Validation accuracy: 0.7941176470588235
[[65  3]
 [25 43]]
```

### Grid search for Linear SVM (SGDClassifier)

In [41]:
pg_lin = {
    'alpha': [0.0005, 0.00075, 0.00100, 0.00500, 0.010, 0.015, 0.025, 0.050, 0.5, 1.0],
    'loss': ['hinge', 'log'],
    'penalty': ['l1', 'l2', 'elasticnet'],
    # The default iteration count is 5.
    'n_iter': [5, 25],
}

# The CV isn't really necessary, nor does it make sense, but there's no real way to avoid it.
gs_lin = GridSearchCV(lin_clf, pg_lin, cv=2, scoring=score_on_validation, n_jobs=-1)

In [42]:
gs_lin.fit(X_train, y_train)

GridSearchCV(cv=2, error_score='raise',
       estimator=SGDClassifier(alpha=0.001, average=False, class_weight='balanced',
       epsilon=0.1, eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'penalty': ['l1', 'l2', 'elasticnet'], 'alpha': [0.0005, 0.00075, 0.001, 0.005, 0.01, 0.015, 0.025, 0.05, 0.5, 1.0], 'n_iter': [5, 25], 'loss': ['hinge', 'log']},
       pre_dispatch='2*n_jobs', refit=True,
       scoring=<function score_on_validation at 0x1101a1f28>, verbose=0)

In [43]:
report(gs_lin.grid_scores_, n_top=50)

[0] Mean validation score: 0.932 (std: 0.006): {'penalty': 'l2', 'alpha': 0.015, 'n_iter': 5, 'loss': 'log'}
[1] Mean validation score: 0.932 (std: 0.000): {'penalty': 'l2', 'alpha': 0.025, 'n_iter': 25, 'loss': 'hinge'}
[2] Mean validation score: 0.932 (std: 0.003): {'penalty': 'l2', 'alpha': 0.005, 'n_iter': 25, 'loss': 'hinge'}
[3] Mean validation score: 0.932 (std: 0.006): {'penalty': 'elasticnet', 'alpha': 0.025, 'n_iter': 5, 'loss': 'log'}
[4] Mean validation score: 0.931 (std: 0.002): {'penalty': 'elasticnet', 'alpha': 0.01, 'n_iter': 25, 'loss': 'hinge'}
[5] Mean validation score: 0.931 (std: 0.002): {'penalty': 'elasticnet', 'alpha': 0.05, 'n_iter': 25, 'loss': 'hinge'}
[6] Mean validation score: 0.931 (std: 0.008): {'penalty': 'elasticnet', 'alpha': 0.015, 'n_iter': 25, 'loss': 'log'}
[7] Mean validation score: 0.929 (std: 0.010): {'penalty': 'elasticnet', 'alpha': 0.005, 'n_iter': 5, 'loss': 'log'}
[8] Mean validation score: 0.929 (std: 0.003): {'penalty': 'l2', 'alpha': 0.0

Sample from train-A only:
    
```
[0] Mean validation score: 0.806 (std: 0.003): {'loss': 'hinge', 'penalty': 'l1', 'alpha': 0.025}
[1] Mean validation score: 0.806 (std: 0.009): {'loss': 'log', 'penalty': 'l1', 'alpha': 0.025}
[2] Mean validation score: 0.794 (std: 0.032): {'loss': 'hinge', 'penalty': 'l1', 'alpha': 0.001}
[3] Mean validation score: 0.794 (std: 0.006): {'loss': 'log', 'penalty': 'elasticnet', 'alpha': 0.075}
[4] Mean validation score: 0.794 (std: 0.000): {'loss': 'hinge', 'penalty': 'elasticnet', 'alpha': 0.1}
[5] Mean validation score: 0.792 (std: 0.014): {'loss': 'hinge', 'penalty': 'l2', 'alpha': 0.075}
[6] Mean validation score: 0.789 (std: 0.017): {'loss': 'hinge', 'penalty': 'elasticnet', 'alpha': 0.05}
[7] Mean validation score: 0.789 (std: 0.014): {'loss': 'hinge', 'penalty': 'l1', 'alpha': 0.1}
[8] Mean validation score: 0.789 (std: 0.003): {'loss': 'log', 'penalty': 'l2', 'alpha': 0.1}
[9] Mean validation score: 0.787 (std: 0.045): {'loss': 'hinge', 'penalty': 'elasticnet', 'alpha': 0.001}
[10] Mean validation score: 0.787 (std: 0.016): {'loss': 'hinge', 'penalty': 'l2', 'alpha': 0.1}
[11] Mean validation score: 0.787 (std: 0.012): {'loss': 'log', 'penalty': 'l2', 'alpha': 0.075}
[12] Mean validation score: 0.787 (std: 0.010): {'loss': 'hinge', 'penalty': 'elasticnet', 'alpha': 0.075}
[13] Mean validation score: 0.787 (std: 0.006): {'loss': 'hinge', 'penalty': 'l2', 'alpha': 0.5}
[14] Mean validation score: 0.784 (std: 0.015): {'loss': 'log', 'penalty': 'elasticnet', 'alpha': 0.1}
[15] Mean validation score: 0.784 (std: 0.017): {'loss': 'log', 'penalty': 'elasticnet', 'alpha': 0.5}
[16] Mean validation score: 0.784 (std: 0.019): {'loss': 'log', 'penalty': 'l1', 'alpha': 0.1}
[17] Mean validation score: 0.782 (std: 0.015): {'loss': 'log', 'penalty': 'l2', 'alpha': 0.5}
[18] Mean validation score: 0.782 (std: 0.014): {'loss': 'log', 'penalty': 'l2', 'alpha': 0.05}
[19] Mean validation score: 0.782 (std: 0.012): {'loss': 'hinge', 'penalty': 'l1', 'alpha': 0.01}
[20] Mean validation score: 0.782 (std: 0.009): {'loss': 'hinge', 'penalty': 'elasticnet', 'alpha': 0.01}
[21] Mean validation score: 0.779 (std: 0.022): {'loss': 'log', 'penalty': 'l2', 'alpha': 0.025}
[22] Mean validation score: 0.779 (std: 0.016): {'loss': 'hinge', 'penalty': 'l2', 'alpha': 0.05}
[23] Mean validation score: 0.779 (std: 0.010): {'loss': 'log', 'penalty': 'elasticnet', 'alpha': 0.05}
[24] Mean validation score: 0.777 (std: 0.018): {'loss': 'hinge', 'penalty': 'elasticnet', 'alpha': 0.025}
[25] Mean validation score: 0.777 (std: 0.024): {'loss': 'hinge', 'penalty': 'l1', 'alpha': 0.075}
[26] Mean validation score: 0.777 (std: 0.017): {'loss': 'log', 'penalty': 'l1', 'alpha': 0.05}
[27] Mean validation score: 0.775 (std: 0.018): {'loss': 'hinge', 'penalty': 'l2', 'alpha': 0.01}
[28] Mean validation score: 0.775 (std: 0.009): {'loss': 'log', 'penalty': 'l2', 'alpha': 0.01}
[29] Mean validation score: 0.767 (std: 0.033): {'loss': 'hinge', 'penalty': 'l2', 'alpha': 0.025}
[30] Mean validation score: 0.767 (std: 0.049): {'loss': 'log', 'penalty': 'l1', 'alpha': 0.075}
[31] Mean validation score: 0.762 (std: 0.021): {'loss': 'log', 'penalty': 'elasticnet', 'alpha': 0.025}
[32] Mean validation score: 0.757 (std: 0.022): {'loss': 'log', 'penalty': 'l1', 'alpha': 0.01}
[33] Mean validation score: 0.755 (std: 0.003): {'loss': 'log', 'penalty': 'elasticnet', 'alpha': 0.01}
[34] Mean validation score: 0.750 (std: 0.037): {'loss': 'log', 'penalty': 'l1', 'alpha': 0.001}
[35] Mean validation score: 0.728 (std: 0.094): {'loss': 'log', 'penalty': 'l2', 'alpha': 0.0005}
[36] Mean validation score: 0.716 (std: 0.050): {'loss': 'log', 'penalty': 'elasticnet', 'alpha': 0.001}
[37] Mean validation score: 0.708 (std: 0.072): {'loss': 'log', 'penalty': 'l2', 'alpha': 0.001}
[38] Mean validation score: 0.708 (std: 0.062): {'loss': 'hinge', 'penalty': 'elasticnet', 'alpha': 0.0005}
[39] Mean validation score: 0.701 (std: 0.106): {'loss': 'log', 'penalty': 'l1', 'alpha': 0.0005}
[40] Mean validation score: 0.686 (std: 0.139): {'loss': 'hinge', 'penalty': 'l1', 'alpha': 0.05}
[41] Mean validation score: 0.686 (std: 0.132): {'loss': 'hinge', 'penalty': 'elasticnet', 'alpha': 0.5}
[42] Mean validation score: 0.672 (std: 0.121): {'loss': 'log', 'penalty': 'elasticnet', 'alpha': 0.0005}
[43] Mean validation score: 0.652 (std: 0.117): {'loss': 'hinge', 'penalty': 'l1', 'alpha': 0.0005}
[44] Mean validation score: 0.652 (std: 0.100): {'loss': 'hinge', 'penalty': 'l2', 'alpha': 0.0005}
[45] Mean validation score: 0.583 (std: 0.191): {'loss': 'hinge', 'penalty': 'l2', 'alpha': 0.001}
[46] Mean validation score: 0.500 (std: 0.000): {'loss': 'hinge', 'penalty': 'l1', 'alpha': 0.5}
[47] Mean validation score: 0.500 (std: 0.000): {'loss': 'log', 'penalty': 'l1', 'alpha': 0.5}
```

In [ ]:
y_pred_lin = gs_lin.predict(X_train)
print(confusion_matrix(y_train, y_pred_lin))

On slightly less data:
```
[[110587   4643]
 [   155   1024]]
 ```

Old:

```
[[70902 10205]
 [  147   607]]
```

Older: 

```
[[63080 18027]
 [  245   509]]
```

## Nonlinear SVC

In [ ]:
nonlinear_svm = SVC(kernel='rbf', class_weight='balanced', C=1)
nonlinear_svm_scores = cross_val_score(nonlinear_svm, X_train, y_train, cv=3)
print(nonlinear_svm_scores)
print(np.mean(nonlinear_svm_scores))

In [ ]:
nonlin_clf = nonlinear_svm
nonlin_clf.fit(X_train, y_train)

In [ ]:
score = nonlin_clf.score(X_valid, y_valid)
y_valid_predicted = nonlin_clf.predict(X_valid)

print("Validation data shape: {0}".format(X_valid.shape))
print("Validation data neg:pos ratio: {0}".format(VALIDATION_NEG_TO_POS_RATIO))
print("Validation accuracy: {0}".format(score))

print(confusion_matrix(y_valid, y_valid_predicted))

Old SVM nonlin numbers:
    
```
Validation data shape: (266, 10)
Validation data neg:pos ratio: 1
Validation accuracy: 0.8045112781954887
[[107  26]
 [ 26 107]]
```

In [ ]:
# Multiple C confusion matrices.

from sklearn.externals.joblib import Parallel, delayed

def quick_eval_svc(C):
    output = ""
    output += "\nC = {0:10.8f}\n--------\n\n".format(C)
    svc = SVC(kernel='rbf', class_weight='balanced', C=C)
    svc.fit(X_train, y_train)
    y_valid_pred = svc.predict(X_valid)
    y_train_pred = svc.predict(X_train)
    output += "Validation data confusion matrix:\n"
    output += str(confusion_matrix(y_valid, y_valid_pred))
    output += "\n"
    output += "Training data confusion matrix (prone to overfitting):\n"
    output += str(confusion_matrix(y_train, y_train_pred))
    output += "\n"
    return output

def confusion_search():
    Cs = [0.00001, 0.0001, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 2, 5]
    res = Parallel(n_jobs=6)(delayed(quick_eval_svc)(C) for C in Cs)
    return res

In [ ]:
results = confusion_search()

In [ ]:
for r in results:
    print(r)

Old confusion matrices (before positive label bug fix):

```
C = 0.00001000
--------

Validation data confusion matrix:
[[133   0]
 [133   0]]
Training data confusion matrix (prone to overfitting):
[[81107     0]
 [  754     0]]


C = 0.00010000
--------

Validation data confusion matrix:
[[105  28]
 [ 25 108]]
Training data confusion matrix (prone to overfitting):
[[67386 13721]
 [  171   583]]


C = 0.00100000
--------

Validation data confusion matrix:
[[111  22]
 [ 18 115]]
Training data confusion matrix (prone to overfitting):
[[68205 12902]
 [  130   624]]


C = 0.00500000
--------

Validation data confusion matrix:
[[120  13]
 [ 16 117]]
Training data confusion matrix (prone to overfitting):
[[70683 10424]
 [  110   644]]


C = 0.01000000
--------

Validation data confusion matrix:
[[120  13]
 [ 16 117]]
Training data confusion matrix (prone to overfitting):
[[70969 10138]
 [  105   649]]


C = 0.05000000
--------

Validation data confusion matrix:
[[120  13]
 [ 15 118]]
Training data confusion matrix (prone to overfitting):
[[72372  8735]
 [   82   672]]


C = 0.10000000
--------

Validation data confusion matrix:
[[121  12]
 [ 18 115]]
Training data confusion matrix (prone to overfitting):
[[72817  8290]
 [   67   687]]


C = 0.50000000
--------

Validation data confusion matrix:
[[123  10]
 [ 25 108]]
Training data confusion matrix (prone to overfitting):
[[74080  7027]
 [   40   714]]


C = 0.60000000
--------

Validation data confusion matrix:
[[124   9]
 [ 29 104]]
Training data confusion matrix (prone to overfitting):
[[74291  6816]
 [   38   716]]


C = 0.70000000
--------

Validation data confusion matrix:
[[123  10]
 [ 30 103]]
Training data confusion matrix (prone to overfitting):
[[74413  6694]
 [   34   720]]


C = 0.80000000
--------

Validation data confusion matrix:
[[124   9]
 [ 30 103]]
Training data confusion matrix (prone to overfitting):
[[74562  6545]
 [   33   721]]


C = 0.90000000
--------

Validation data confusion matrix:
[[124   9]
 [ 30 103]]
Training data confusion matrix (prone to overfitting):
[[74678  6429]
 [   32   722]]


C = 1.00000000
--------

Validation data confusion matrix:
[[125   8]
 [ 30 103]]
Training data confusion matrix (prone to overfitting):
[[74798  6309]
 [   32   722]]


C = 2.00000000
--------

Validation data confusion matrix:
[[126   7]
 [ 38  95]]
Training data confusion matrix (prone to overfitting):
[[75630  5477]
 [   24   730]]


C = 5.00000000
--------

Validation data confusion matrix:
[[128   5]
 [ 48  85]]
Training data confusion matrix (prone to overfitting):
[[76552  4555]
 [   13   741]]
```

### Grid search for kernelized SVM (expensive to compute)

In [ ]:
pg_nonlin = {
    'C': [0.001, 0.01, 0.075, 0.100, 0.125, 0.5, 0.75, 0.9, 1, 1.2],
}

gs_nonlin = GridSearchCV(nonlin_clf, pg_nonlin, cv=2, scoring=score_on_validation, verbose=2, n_jobs=4)

In [ ]:
gs_nonlin.fit(X_train, y_train)

Top results from nonlinear SVM grid search (rbf kernel, default settings, checked on 15% validation data set, biased for 1:1 pos/neg labels).
Using 10 features.


```
[0] Mean validation score: 0.824 (std: 0.010): {'C': 0.0005}
[1] Mean validation score: 0.811 (std: 0.015): {'C': 0.001}
[2] Mean validation score: 0.782 (std: 0.015): {'C': 0.01}
[3] Mean validation score: 0.748 (std: 0.007): {'C': 0.1}
[4] Mean validation score: 0.733 (std: 0.003): {'C': 0.05}
[5] Mean validation score: 0.694 (std: 0.012): {'C': 0.5}
[6] Mean validation score: 0.691 (std: 0.010): {'C': 0.75}
```

Top results from nonlinear SVM grid search (rbf kernel, default settings, ...).
Using 20 features.

```
[0] Mean validation score: 0.860 (std: 0.008): {'C': 0.01}
[1] Mean validation score: 0.848 (std: 0.010): {'C': 0.1}
[2] Mean validation score: 0.840 (std: 0.004): {'C': 0.001}
[3] Mean validation score: 0.836 (std: 0.005): {'C': 0.0005}
[4] Mean validation score: 0.796 (std: 0.004): {'C': 1}
```

In [ ]:
report(gs_nonlin.grid_scores_, n_top=250)

In [ ]:
y_pred_nonlin = gs_nonlin.predict(X_train)
print(confusion_matrix(y_train, y_pred_nonlin))

y_pred_valid_nonlin = gs_nonlin.predict(X_valid)
print(confusion_matrix(y_valid, y_pred_valid_nonlin))

## Old metrics code

In [ ]:
def train_metrics(clf, X, y):
    clf.fit(X, y)
    y_pred = clf.predict(X)
    print(average_precision_score(y, y_pred, average='micro'))
    print(average_precision_score(y, y_pred, average='macro'))
    print(average_precision_score(y, y_pred, average='weighted'))
#     print("F1:        {0:.4f}".format(f1_score(y, y_pred, pos_label='true\n')))
#     print("Precision: {0:.4f}".format(precision_score(y, y_pred)))
#     print("Accuracy:  {0:.4f}".format(accuracy_score(y, y_pred)))
    print("Confusion matrix:")
    print(confusion_matrix(y, y_pred, labels=None))

In [ ]:
# pred_y = clf.predict(X)
#clf = SGDClassifier(class_weight='balanced')
train_metrics(LinearSVC(class_weight='balanced'), X, y_raw)

In [ ]:
train_metrics(SVC(kernel='rbf', class_weight='balanced'), X, y_raw)